In [5]:
pip install cairocffi

     |████████████████████████████████| 86 kB 711 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for cairocffi: filename=cairocffi-1.5.0-py3-none-any.whl size=90524 sha256=65f4c82d00776043a0c79d6ea1ff55ac363367a39a7b57daa0f9ce3752404542
  Stored in directory: /home/user201/.cache/pip/wheels/34/c0/21/58ab88c0e5a36a32f10f5257afbd6e5fae805171da13691863
Successfully built cairocffi
Note: you may need to restart the kernel to use updated packages.


In [69]:
def calculate_region_centroid(vor, region):
    polygon = [vor.vertices[vertex_idx] for vertex_idx in region]
    area = 0.0
    centroid_x = 0.0
    centroid_y = 0.0

    for i in range(len(polygon)):
        p1 = polygon[i]
        p2 = polygon[(i + 1) % len(polygon)]

        cross_product = p1[0] * p2[1] - p2[0] * p1[1]
        area += cross_product
        centroid_x += (p1[0] + p2[0]) * cross_product
        centroid_y += (p1[1] + p2[1]) * cross_product

    area *= 0.5
    centroid_x /= (6.0 * area)
    centroid_y /= (6.0 * area)

    return centroid_x, centroid_y


In [126]:
import numpy as np
from scipy.spatial import Voronoi
from noise import snoise2
import cairocffi as cairo

width, height = 800, 600
num_points = 1000
octaves = 6
freq = 32.0 * octaves

points = np.random.randint(0, width, (num_points, 2))
vor = Voronoi(points)

def generate_noise(width, height, seed=1337):
    world = np.zeros((height, width))

    for y in range(height):
        for x in range(width):
            n = snoise2(x / freq, y / freq, octaves, base=seed)
            n = (n + 1) / 2  # normalize noise values between 0 and 1
            world[y, x] = n

    return world

terrain_noise = generate_noise(width, height, seed=1)


def assign_terrain(vor, terrain_noise):
    terrain = []
    height, width = terrain_noise.shape
    for idx, region in enumerate(vor.regions):
        if not region:  # Ignore empty regions or those outside the bounding box
            terrain.append('water')
            continue
        elif -1 in region:
            terrain.append(None)
            continue
            
        centroid = calculate_region_centroid(vor, region)
        x, y = centroid

        # Ensure coordinates are within bounds
        x = min(max(0, int(x)), width - 1)
        y = min(max(0, int(y)), height - 1)

        terrain_type = "land" if terrain_noise[y, x] < 0.5 else "water"
        terrain.append(terrain_type)
    return terrain


def draw_map(vor, terrain, width, height):
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, width, height)
    ctx = cairo.Context(surface)

    # Fill the blank regions on the edges with water
    ctx.set_source_rgb(0.1, 0.4, 0.8)
    ctx.rectangle(0, 0, width, height)
    ctx.fill()

    terrain_colors = {
        "land": (0.5, 0.8, 0.3),
        "water": (0.1, 0.4, 0.8)
    }

    for idx, region in enumerate(vor.regions):
        if not region or idx not in terrain:
            continue

        ctx.set_source_rgb(*terrain_colors[terrain[idx]])

        vertices = vor.vertices[region + [region[0]]]  # close the polygon

        for i, (x, y) in enumerate(vertices):
            if i == 0:
                ctx.move_to(x, y)
            else:
                ctx.line_to(x, y)
        ctx.close_path()
        ctx.fill()

    surface.write_to_png("world_map.png")

draw_map(vor, terrain, width, height)


In [127]:
terrain_noise

array([[0.27986616, 0.29547586, 0.31467727, ..., 0.37189168, 0.36566104,
        0.36085173],
       [0.2700171 , 0.28834356, 0.30998074, ..., 0.3770101 , 0.3696035 ,
        0.3630597 ],
       [0.26423363, 0.28231853, 0.30405512, ..., 0.38166214, 0.37502161,
        0.36958726],
       ...,
       [0.68015438, 0.67051551, 0.6627354 , ..., 0.599155  , 0.59453312,
        0.59149256],
       [0.68242145, 0.67194583, 0.66237457, ..., 0.6078361 , 0.60427017,
        0.59968378],
       [0.68843545, 0.67578939, 0.6633008 , ..., 0.61339504, 0.60962397,
        0.60388225]])

In [128]:
import networkx as nx
from scipy.spatial.distance import pdist, squareform

def add_towns_and_roads(vor, terrain, width, height, num_towns=50, road_color=(0.2, 0.2, 0.2), town_color=(0.9, 0.1, 0.1)):
    def find_containing_region(vor, point):
        for idx, region in enumerate(vor.regions):
            if not region:
                continue
            vertices = vor.vertices[region]
            path = mplPath.Path(vertices)
            if path.contains_point(point):
                return idx
        return None

    town_points = []
    for _ in range(num_towns):
        while True:
            x, y = np.random.randint(0, width), np.random.randint(0, height)
            region_idx = find_containing_region(vor, (x, y))
            if region_idx is not None and (terrain[region_idx] == "land"):
                town_points.append((x, y))
                break

    # Create a minimum spanning tree to connect the towns with roads
    town_graph = nx.Graph()
    for i, town1 in enumerate(town_points):
        for j, town2 in enumerate(town_points[i + 1:], start=i + 1):
            distance = np.linalg.norm(np.array(town1) - np.array(town2))
            town_graph.add_edge(i, j, weight=distance)
    mst = nx.minimum_spanning_tree(town_graph)

    def draw_towns_and_roads(ctx):
        # Draw roads
        ctx.set_source_rgb(*road_color)
        ctx.set_line_width(2)
        for edge in mst.edges():
            x1, y1 = town_points[edge[0]]
            x2, y2 = town_points[edge[1]]
            region_idx1 = find_containing_region(vor, (x1, y1))
            region_idx2 = find_containing_region(vor, (x2, y2))
            if region_idx1 is not None and terrain[region_idx1] == "land" and region_idx2 is not None and terrain[region_idx2] == "land":
                ctx.move_to(x1, y1)
                ctx.line_to(x2, y2)
                ctx.stroke()

        # Draw towns
        ctx.set_source_rgb(*town_color)
        for x, y in town_points:
            ctx.arc(x, y, 5, 0, 2 * math.pi)
            ctx.fill()

    return draw_towns_and_roads



In [129]:
def draw_map(vor, terrain, width, height, num_towns=50):
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, width, height)
    ctx = cairo.Context(surface)

    # Draw the background
    ctx.rectangle(0, 0, width, height)
    ctx.set_source_rgb(0.5, 0.5, 0.5)
    ctx.fill()

    # Draw the Voronoi regions
    for idx, region in enumerate(vor.regions):
        if not region:  # Empty region
            continue

        if terrain[idx] == "water":
            ctx.set_source_rgb(0, 0, 1)
        else:
            ctx.set_source_rgb(0, 1, 0)

        for i in range(len(region)):
            p1 = vor.vertices[region[i]]
            p2 = vor.vertices[region[(i + 1) % len(region)]]
            ctx.move_to(*p1)
            ctx.line_to(*p2)
            ctx.stroke()

        polygon = [vor.vertices[vertex_idx] for vertex_idx in region]
        ctx.move_to(*polygon[0])
        for point in polygon[1:]:
            ctx.line_to(*point)
        ctx.close_path()
        ctx.fill()

    draw_towns_and_roads = add_towns_and_roads(vor, terrain, width, height, num_towns=50)
    draw_towns_and_roads(ctx)

    surface.write_to_png("world_map.png")


In [130]:
def find_containing_region(vor, point):
    for idx, region in enumerate(vor.regions):
        if not region or idx not in terrain:
            continue
        vertices = vor.vertices[region]
        path = mplPath.Path(vertices)
        if path.contains_point(point):
            return idx
    return None


In [131]:
from matplotlib import path as mplPath


In [133]:
# Set the desired map dimensions
new_width = 800
new_height = 600
num_points = 2000

# Generate a new Voronoi diagram
new_points = np.random.randint(0, width, (num_points, 2))
new_vor = Voronoi(new_points)

# Generate new terrain and terrain noise
new_terrain_noise = generate_noise(new_width, new_height)
new_terrain = assign_terrain(new_vor, new_terrain_noise)

# Create and save the new map
draw_map(new_vor, new_terrain, new_width, new_height, num_towns=50)


In [60]:
terrain

{2: 'water',
 3: 'water',
 5: 'water',
 8: 'water',
 10: 'land',
 12: 'water',
 13: 'water',
 14: 'water',
 16: 'water',
 17: 'water',
 22: 'land',
 23: 'land',
 24: 'land',
 26: 'land',
 28: 'water',
 30: 'water',
 31: 'water',
 32: 'land',
 33: 'land',
 34: 'land',
 35: 'land',
 36: 'land',
 37: 'land',
 38: 'land',
 39: 'water',
 40: 'water',
 41: 'land',
 42: 'land',
 43: 'land',
 44: 'water',
 45: 'water',
 46: 'water',
 47: 'water',
 48: 'water',
 49: 'water',
 50: 'water',
 51: 'land',
 52: 'land',
 53: 'water',
 54: 'land',
 55: 'water',
 56: 'water',
 57: 'land',
 58: 'land',
 59: 'land',
 60: 'land',
 61: 'water',
 62: 'land',
 66: 'water',
 67: 'land',
 68: 'land',
 69: 'land',
 70: 'water',
 71: 'land',
 72: 'water',
 73: 'land',
 75: 'water',
 76: 'water',
 77: 'water',
 78: 'land',
 79: 'water',
 92: 'land',
 93: 'land',
 94: 'land',
 96: 'water',
 97: 'water',
 98: 'water',
 99: 'water',
 100: 'land',
 101: 'land',
 102: 'water',
 103: 'land',
 108: 'water',
 109: 'water

In [8]:
terrain

{2: 'land',
 6: 'land',
 7: 'water',
 9: 'land',
 13: 'land',
 15: 'water',
 16: 'land',
 17: 'land',
 18: 'water',
 19: 'water',
 20: 'water',
 21: 'land',
 22: 'water',
 24: 'water',
 25: 'water',
 29: 'land',
 30: 'land',
 31: 'land',
 32: 'land',
 33: 'land',
 34: 'land',
 35: 'water',
 36: 'land',
 37: 'water',
 38: 'land',
 40: 'land',
 41: 'land',
 42: 'water',
 43: 'water',
 44: 'water',
 45: 'water',
 47: 'water',
 48: 'land',
 49: 'land',
 50: 'land',
 51: 'water',
 52: 'land',
 54: 'land',
 55: 'land',
 56: 'water',
 57: 'water',
 60: 'water',
 61: 'land',
 62: 'land',
 63: 'land',
 64: 'land',
 65: 'water',
 66: 'land',
 67: 'land',
 68: 'land',
 69: 'water',
 70: 'water',
 71: 'land',
 72: 'water',
 73: 'water',
 75: 'water',
 77: 'land',
 78: 'water',
 79: 'land',
 80: 'land',
 92: 'land',
 93: 'land',
 94: 'land',
 95: 'land',
 96: 'land',
 97: 'land',
 98: 'water',
 99: 'land',
 100: 'land',
 101: 'land',
 102: 'water',
 103: 'land',
 104: 'land',
 105: 'land',
 106: 'w